# ДЗ2 Вариант 2
## Познянский Михаил ИУ6-54Б

  1. Оцените дистанцию поездок (в метрах) на основе координат начальной и конечной станций (без учета поездок, завершившихся там же где и начались)
    
  2. Выведите максимальное, среднее значение, стандартное отклонение и медиан



In [15]:
!pip install pyspark

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType

# Скачиваем датасет
!wget https://s3.amazonaws.com/tripdata/JC-201902-citibike-tripdata.csv.zip

!unzip -o JC-201902-citibike-tripdata.csv.zip

--2026-01-07 19:57:52--  https://s3.amazonaws.com/tripdata/JC-201902-citibike-tripdata.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.234.192, 52.217.97.230, 16.15.199.50, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.234.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491166 (480K) [application/zip]
Saving to: ‘JC-201902-citibike-tripdata.csv.zip.2’

JC-201902-citibike- 100%[===================>] 479.65K  --.-KB/s    in 0.02s   

2026-01-07 19:57:52 (20.4 MB/s) - ‘JC-201902-citibike-tripdata.csv.zip.2’ saved [491166/491166]

Archive:  JC-201902-citibike-tripdata.csv.zip
  inflating: JC-201902-citibike-tripdata.csv  


In [16]:
# Создаем сессию
spark = SparkSession.builder \
    .appName("HW2_Distance_Analysis") \
    .master("local[*]") \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("JC-201902-citibike-tripdata.csv")

df.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- start station id: integer (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: integer (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: double (nullable = true)
 |-- end station longitude: double (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: integer (nullable = true)
 |-- gender: integer (nullable = true)



### Пункт 1

In [19]:
# Радиус Земли в метрах
r = 6371000

# Убираем поездки, где ID начальной и конечной станции совпадают
filtered_df = df.where(F.col("start station id") != F.col("end station id"))

start_lat = F.radians(F.col("start station latitude"))
start_lon = F.radians(F.col("start station longitude"))
end_lat = F.radians(F.col("end station latitude"))
end_lon = F.radians(F.col("end station longitude"))

# Реализация формулы Хаверсина
part1 = F.pow(F.sin((end_lat - start_lat) / 2), 2)
part2 = F.cos(start_lat) * F.cos(end_lat) * F.pow(F.sin((end_lon - start_lon) / 2), 2)
a = part1 + part2
d = 2 * r * F.atan2(F.sqrt(a), F.sqrt(1 - a))

result_df = filtered_df.withColumn("distance", d)

result_df.select("start station name", "end station name", "distance").show(10, truncate=False)

+------------------+----------------+------------------+
|start station name|end station name|distance          |
+------------------+----------------+------------------+
|Exchange Place    |Harborside      |340.4460536411022 |
|Exchange Place    |Grand St        |375.3709380733173 |
|Exchange Place    |Paulus Hook     |233.80784165827936|
|Exchange Place    |Newark Ave      |1231.457292616472 |
|Exchange Place    |Manila & 1st    |996.0025566461777 |
|Exchange Place    |Essex Light Rail|462.80930603096357|
|Exchange Place    |Grove St PATH   |894.6975710017139 |
|Exchange Place    |Grove St PATH   |894.6975710017139 |
|Exchange Place    |Monmouth and 6th|1664.6010123761082|
|Exchange Place    |Grand St        |375.3709380733173 |
+------------------+----------------+------------------+
only showing top 10 rows


### Пункт 2

In [21]:
# Получаем нужные значения
stats = result_df.select(
    F.max("distance").alias("max_distance"),
    F.mean("distance").alias("mean_distance"),
    F.stddev("distance").alias("stddev_distance"),
    F.percentile_approx("distance", 0.5).alias("median_distance")
)

print("Statistics for distances (in meters):")
stats.show(truncate=False)

Statistics for distances (in meters):
+-----------------+-----------------+-----------------+-----------------+
|max_distance     |mean_distance    |stddev_distance  |median_distance  |
+-----------------+-----------------+-----------------+-----------------+
|4975.170135370611|963.2505212560991|570.8997349563779|829.7748149644184|
+-----------------+-----------------+-----------------+-----------------+

